In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [ ]:
df = pd.read_csv("/kaggle/input/kaggle-wars-eclipse/train (1).csv")
df_test = pd.read_csv("/kaggle/input/kaggle-wars-eclipse/test (2).csv")

In [ ]:
df.head()

In [ ]:
df_1, df_2, df_3, df_4, df_5 = [x for _, x in df.groupby(df['Rating'])]

In [ ]:
df_1.to_csv("one.csv")
df_2.to_csv("two.csv")
df_3.to_csv("three.csv")
df_4.to_csv("four.csv")
df_5.to_csv("five.csv")

In [ ]:
df_1.shape, df_2.shape, df_3.shape, df_4.shape, df_5.shape

In [ ]:
df_1["class"] = 1
df_2["class"] = 2
df_3["class"] = 3
df_4["class"] = 4
df_5["class"] = 5

In [ ]:
df_merge = pd.concat([df_1, df_2, df_3, df_4, df_5], axis =0)
df_merge.head(10)

In [ ]:
df_merge.columns

In [ ]:
df_merge_drop = df_merge.drop(["Rating"], axis=1)

In [ ]:
df_merge_drop.isnull().sum()

In [ ]:
df_merge_drop = df_merge_drop.sample(frac = 1)
df_merge_drop.head(10)

In [ ]:
df_merge_drop.reset_index(inplace = True)
df_merge_drop.drop(["index"], axis = 1, inplace = True)

In [ ]:
df_merge_drop.columns
df_merge_drop.head()

In [ ]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [ ]:
from string import digits
for i in range(0,len(df_merge_drop)):
    remove_digits = str.maketrans('', '', digits)
    ini_string=str(df_merge_drop["Review"][i])
    df_merge_drop["Review"][i] = ini_string.translate(remove_digits)

In [ ]:
import re
#df_merge_drop["text"] = df_merge_drop["text"].apply(numericDelete)
df_test["Review"] = df_test["Review"].apply(wordopt)

In [ ]:
x = df_merge_drop["Review"]
y = df_merge_drop["class"]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier


RFC = RandomForestClassifier(random_state=0)
RFC.fit(xv_train, y_train)

In [ ]:
pred_rfc = RFC.predict(xv_test)

In [ ]:
RFC.score(xv_test, y_test)

In [ ]:
from sklearn import svm

SVM = svm.SVC()
SVM.fit(xv_train, y_train)

In [ ]:
pred_svm = SVM.predict(xv_test)

In [ ]:
SVM.score(xv_test, y_test)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

In [ ]:
pred_dt = DT.predict(xv_test)

In [ ]:
DT.score(xv_test, y_test)

In [ ]:
# path to file you will use for predictions
test_data_path = '/kaggle/input/kaggle-wars-eclipse/test (2).csv'

# read test data file using pandas
test_data = pd.read_csv(test_data_path)

import re
test_data["Review"] = test_data["Review"].apply(wordopt)

from sklearn.feature_extraction.text import TfidfVectorizer

test_X = test_data["Review"]
vectorization1 = TfidfVectorizer()
xv1_test = vectorization.transform(test_X)

# create test_X which comes from test_data but includes only the columns you used for prediction.
# The list of columns is stored in a variable called features

ids = list(range(7000))

# make predictions which we will submit. 
test_preds_RF = RFC.predict(xv1_test)
test_preds_DT = DT.predict(xv1_test)
test_preds_SVM = SVM.predict(xv1_test)

# The lines below shows you how to save your data in the format needed to score it in the competition
output_RF = pd.DataFrame({'Id': ids , 'Rating': test_preds_RF})
output_DT = pd.DataFrame({'Id': ids , 'Rating': test_preds_DT})
output_SVM = pd.DataFrame({'Id': ids , 'Rating': test_preds_SVM})

output_RF.to_csv('submission_RF.csv', index=False)
output_DT.to_csv('submission_DT.csv', index=False)
output_SVM.to_csv('submission_SVM.csv', index=False)